In [11]:
from sentence_transformers import SentenceTransformer, util

In [12]:
import pandas as pd

df = pd.read_csv('data.csv')

question = df['Question']
answer = df['Answer']

In [13]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [14]:
embeddings = model.encode(question)

In [5]:
from FlagEmbedding import FlagReranker

reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)


In [54]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [56]:
tokenizerId = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-id-en")
modelId = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-id-en")


c:\Program Files\Python38\lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [57]:
tokenizerEnId = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-id")
modelEnId = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-id")

In [25]:
tokenizerInc = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-inc-en")
modelInc = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-inc-en")

c:\Program Files\Python38\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dao Nam\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-inc-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Program Files\Python38\lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarnin

In [108]:
tokenizerEnInc = AutoTokenizer.from_pretrained("thilina/mt5-sinhalese-english")
modelEnInc = AutoModelForSeq2SeqLM.from_pretrained("thilina/mt5-sinhalese-english")

In [26]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizerVi = AutoTokenizer.from_pretrained("VietAI/envit5-translation")
modelVi = AutoModelForSeq2SeqLM.from_pretrained("VietAI/envit5-translation")

c:\Program Files\Python38\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dao Nam\.cache\huggingface\hub\models--VietAI--envit5-translation. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
input_ids = tokenizerEnInc("How are you", return_tensors="pt").input_ids
outputs = modelEnInc.generate(input_ids=input_ids)
translatedSentence = tokenizerEnInc.batch_decode(outputs, skip_special_tokens=True)

In [74]:
def translateToEng(sentence, originLanguage):
    if (originLanguage == 'English'):
        return sentence
    elif (originLanguage == 'Vietnamese'):
        input_ids = tokenizerVi(sentence, return_tensors="pt",padding=True).input_ids
        outputs = modelVi.generate(input_ids=input_ids, max_length=512)
        translatedSentence = tokenizerVi.batch_decode(outputs, skip_special_tokens=True)
        return translatedSentence[0][4:]
    elif (originLanguage == 'Indonesian'):
        input_ids = tokenizerId(sentence, return_tensors="pt",padding=True).input_ids
        outputs = modelId.generate(input_ids=input_ids, max_length=512)
        translatedSentence = tokenizerId.batch_decode(outputs, skip_special_tokens=True)
        return translatedSentence[0]
    elif (originLanguage == 'Sinhala'):
        input_ids = tokenizerInc(sentence, return_tensors="pt",padding=True).input_ids
        outputs = modelInc.generate(input_ids=input_ids, max_length=512)
        translatedSentence = tokenizerInc.batch_decode(outputs, skip_special_tokens=True)
        return translatedSentence[0]

In [110]:
def translateFromEng(sentence, originLanguage):
    if (originLanguage == 'English'):
        return sentence
    elif (originLanguage == 'Vietnamese'):
        input_ids = tokenizerVi(sentence, return_tensors="pt",padding=True).input_ids
        outputs = modelVi.generate(input_ids=input_ids, max_length=512)
        translatedSentence = tokenizerVi.batch_decode(outputs, skip_special_tokens=True)
        return translatedSentence[0][4:]
    elif (originLanguage == 'Indonesian'):
        input_ids = tokenizerEnId(sentence, return_tensors="pt",padding=True).input_ids
        outputs = modelEnId.generate(input_ids=input_ids, max_length=512)
        translatedSentence = tokenizerEnId.batch_decode(outputs, skip_special_tokens=True)
        return translatedSentence[0]
    elif (originLanguage == 'Sinhala'):
        input_ids = tokenizerEnInc(sentence, return_tensors="pt",padding=True).input_ids
        outputs = modelEnInc.generate(input_ids=input_ids, max_length=512)
        translatedSentence = tokenizerEnInc.batch_decode(outputs, skip_special_tokens=True)
        return translatedSentence[0]

In [113]:
def return_answer(user_question, qLanguage, aLanguage):
    user_question = translateToEng(user_question, qLanguage)

    q_embedding = model.encode(user_question)

    cos_sim = util.cos_sim(q_embedding, embeddings)

    val = []

    for i in range(len(cos_sim[0])):
        val.append(cos_sim[0][i].item())

    p_answer = answer.copy()
    p_question = question.copy()

    for i in range(len(val)):
        for j in range(i + 1, len(val)):
            if (val[i] < val[j]):
                temp = val[i]
                val[i] = val[j]
                val[j] = temp

                temp = p_answer[i]
                p_answer[i] = p_answer[j]
                p_answer[j] = temp
                temp = p_question[i]
                p_question[i] = p_question[j]
                p_question[j] = temp

    answer1 = "Sorry we don't know the answer"
    answer2 = ""
    answer3 = ""
    if (reranker.compute_score([p_question[0], user_question]) >= -6):
        answer1 = translateFromEng(p_answer[0], aLanguage)
    if (reranker.compute_score([p_question[1], user_question]) >= -6):
        answer2 = translateFromEng(p_answer[1], aLanguage)
    if (reranker.compute_score([p_question[2], user_question]) >= -6):
        answer3 = translateFromEng(p_answer[2], aLanguage)

    return (answer1, answer2, answer3)

translateFromEng("Hello","Sinhala");
    

In [99]:
import gradio as gr # type: ignore
    
demo = gr.Interface(
    fn=return_answer,
    inputs=[
        gr.Textbox(label="Question"),
        gr.Dropdown(
            ["English", "Indonesian", "Vietnamese", "Sinhala"], label="Question language", value="English"
        ),
        gr.Dropdown(
            ["English", "Indonesian", "Vietnamese", "Sinhala"], label="Answer language", value="English"
        )
    ],
    outputs=[gr.Textbox(label="Answer 1"), gr.Textbox(label="Answer 2"), gr.Textbox(label="Answer 3")]
)

demo.launch()

C:\Users\Dao Nam\AppData\Roaming\Python\Python38\site-packages\gradio\utils.py:977: UserWarning: Expected 2 arguments for function <function return_answer at 0x0000025F1A4170D0>, received 3.
  warnings.warn(
C:\Users\Dao Nam\AppData\Roaming\Python\Python38\site-packages\gradio\utils.py:985: UserWarning: Expected maximum 2 arguments for function <function return_answer at 0x0000025F1A4170D0>, received 3.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\Dao Nam\AppData\Roaming\Python\Python38\site-packages\gradio\queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
  File "C:\Users\Dao Nam\AppData\Roaming\Python\Python38\site-packages\gradio\route_utils.py", line 270, in call_process_api
    output = await app.get_blocks().process_api(
  File "C:\Users\Dao Nam\AppData\Roaming\Python\Python38\site-packages\gradio\blocks.py", line 1887, in process_api
    result = await self.call_function(
  File "C:\Users\Dao Nam\AppData\Roaming\Python\Python38\site-packages\gradio\blocks.py", line 1472, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\Dao Nam\AppData\Roaming\Python\Python38\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "C:\Users\Dao Nam\AppData\Roaming\Python\Python38\site-packages\anyio\_backends\_asyncio.py", line 2144, in run_sy

In [ ]:
import tkinter as tk

def change_text():
    user_question = entry.get()
    q_embedding = model.encode(user_question)

    cos_sim = util.cos_sim(q_embedding, embeddings)

    val = []

    for i in range(len(cos_sim[0])):
        val.append(cos_sim[0][i].item())

    p_answer = answer.copy()
    p_question = question.copy()

    for i in range(len(val)):
        for j in range(i + 1, len(val)):
            if (val[i] < val[j]):
                temp = val[i]
                val[i] = val[j]
                val[j] = temp

                temp = p_answer[i]
                p_answer[i] = p_answer[j]
                p_answer[j] = temp
                
                temp = p_question[i]
                p_question[i] = p_question[j]
                p_question[j] = temp

    if (reranker.compute_score([p_question[0], user_question]) > 0):
        answer_label.config(text=p_answer[0])
    else:
        answer_label.config(text="Sorry, we don't know the answer for this question")


window = tk.Tk()
window.geometry('520x300')
label = tk.Label(text="FAQ")
label.pack()

entry = tk.Entry(fg="black", bg="white", width=50)
entry.pack()

# Create a button
button = tk.Button(window, text="Find", command=change_text)
button.pack()

noti_label = tk.Label(text="Answer")
noti_label.pack()

answer_label = tk.Label(text="", wraplength=300, justify='center')
answer_label.pack()

# Run the main event loop
window.mainloop()